In [ ]:
#установка необходимых библиотек bertopic
!pip3 install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 5.7 MB/s eta 0:00:00


In [ ]:
#импортирование библиотек
import pandas as pd
import numpy as np
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


In [16]:
from collections import Counter

In [29]:
#загрузка датесета
df = pd.read_excel('/content/Тесты.xlsx')
docs = df['39 ПРИЧИНА'].tolist()

In [ ]:
#модель для эмбендингов
sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#настройки
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [ ]:
#прописываем UMAP
umap_model = UMAP(
    n_neighbors=15,           # Уменьшено для лучшего локального разделения
    n_components=30,         # Сохраняет больше семантики
    min_dist=0.3,           # Более равномерное распределение
    metric='cosine',
    random_state=42
)
#прописываем HDBSCAN
hdbscan_model = HDBSCAN(
    min_cluster_size=5,     # Средний размер для стабильных кластеров
    min_samples=5,          # Баланс между шумом и кластерами
    #cluster_selection_epsilon=0.1,  # Объединяет близкие группы
    metric='euclidean',
    prediction_data=True
)
#сама модель
topic_model = BERTopic(
    embedding_model=sentence_model, language="russian",
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    min_topic_size=8,       # Разрешает более мелкие, но значимые темы
    n_gram_range=(1, 2),
    top_n_words=10,
    calculate_probabilities=True
)

In [30]:
topics, probs = topic_model.fit_transform(docs)
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,75,-1_цен_они_хотят_что,"[цен, они, хотят, что, не, недовольны, быть, не хотят, что они, то]","[Наверное, повышение цен, зарплату мало платят врачам и другим, президента они что ли хотят менять, Ну люди всегда чем-то недовольны, может быть, политикой государства, а может быть, повышением цен, мало ли, Потому что они смотрят на ситуацию что происходит в целом акции протеста что посадили их президента Саакашвили плохо что они творят хотят чтоб у них с Украиной произошло виорой майдан]"
1,0,762,0_но но_но_предложили но_предложили,"[но но, но, предложили но, предложили, проблем но, що, нко предложили, но що, но нко, но нет]","[НО, НО, НО]"
2,1,184,1_зо_зо зо_сша зо_зо сша,"[зо, зо зо, сша зо, зо сша, иногенты, иногенты зо, зо иногенты, сша, зо бюджет, абхазия]","[ЗО, ЗО, ЗО]"
3,2,109,2_америка_америки_сша_американцы,"[америка, америки, сша, американцы, сша америка, америке, виновата, вмешательство, это, америка америка]","[Америка, Америка, Америка]"
4,3,52,3_провокация_там_то_туда,"[провокация, там, то, туда, сша, потому что, потому, майдан, сторон, провокация сша]","[Провокация сша, Не обьясню у них там свое мутят крутят свои права качают, Там провокация]"
5,4,44,4_не_понятия_без_сказать,"[не, понятия, без, сказать, незнаю, без понятия, знаю, не знаю, не помню, помню]","[Не могу сказать, Не могу сказать, Не могу сказать]"
6,5,36,5_украине_на украине_как_на,"[украине, на украине, как, на, украиной, как на, связано, украина, же, что]","[Хотят такой же переворот как на Украине произвести, Америка хочет сделать переворот как на Украине, Хотят революцию как на украине]"
7,6,33,6_россии_против россии_против_россией,"[россии, против россии, против, россией, не, какой то, россии против, какой, фронт против, россия]","[Открыть фронт против России, Против России, Против России]"
8,7,30,7_вмешательство_вмешательство сша_стран_извне,"[вмешательство, вмешательство сша, стран, извне, извне вмешательство, из вне, вне, сша, влияние, внешнее вмешательство]","[Вмешательство сша, Вмешательство сша, Вмешательство сша]"
9,8,28,8_запада_запад_влияние запада_влияние,"[запада, запад, влияние запада, влияние, западная, запада запад, вмешательство запада, запад влияние, сша западная, запада влияние]","[Влияние Запада, Влияние запада, Влияние запада]"


In [31]:
df_after_analiz = pd.DataFrame({"Document": docs, "Topic": topics}) #создаем датафрейм для просмотра кластеров

In [53]:
topic_n = -1 #id класса
documents_in_topic_n = df_after_analiz[df_after_analiz['Topic'] == topic_n]['Document'].tolist()
documents_in_topic_n

['Фашизм',
 'Потому, что они не хотят жить с президентом Путиным',
 'Должны быть независимые сми',
 'Они не хотят идти нашей дорогой',
 'Недовольство некоторыми экономическими решениями',
 'Провокация со стороны Европпы и Евросоюза',
 'Социальное положение жителей, внешнее влияние',
 'Господи уважение к друг другу нет',
 'Из-за плохого экономического состояния у них',
 'Пенсионный возраст',
 'Ну люди всегда чем-то недовольны, может быть, политикой государства, а может быть, повышением цен, мало ли',
 'Недовольство народа жизнью. Правительством все время недовольны',
 'Не прислушалась путем, вот во Франции были насчст пенсии протесты',
 'Они не хотят быть в зависимости от Европы',
 'Наверное, повышение цен, зарплату мало платят врачам и другим, президента они что ли хотят менять',
 'США народ баламутит, чтобы против России выступать за деньги',
 'Не согласны с политическим усроем',
 'Местные неонайисты',
 'Местный конфликт',
 'Рост цен',
 'Благосостояние граждан',
 'Защита прав людей',


In [28]:
# РУЧНОЕ СЕМАНТИЧЕСКОЕ СОПОСТАВЛЕНИЕ (ОДИН КОД НА ТЕМУ)
manual_single_code_mapping = {
    -1: 11,      # "очень_это_турция землетрясение_мобилизацию" → выбираем спецоперацию
    0: 510,      # "зо_зо зо_зо" → затрудняюсь ответить
    1: 510,      # "ничего ничего_ничего_такого" → затрудняюсь ответить
    2: 250,      # "землетрясение землетрясение" → землетрясение
    3: 11,       # "украине_на украине_на_события на" → спецоперация
    4: 11,       # "война война" → спецоперация
    5: 510,      # "нет нет_нет_не следил" → затрудняюсь ответить
    6: 11,       # "сво сво_сво_лига чемпионов_лига" → спецоперация (основная тема)
    7: 250,      # "турции землетрясение" → землетрясение
    8: 11,       # "украина украина" → спецоперация
    9: 11,       # "украине война" → спецоперация
    10: 510,     # "но_но но_ее знаю" → затрудняюсь ответить
    11: 11,      # "сво сво_сво_всо" → спецоперация
    12: 250,     # "турция_турция" → землетрясение
    13: 510,     # "никакие_никакие" → затрудняюсь ответить
    14: 150,     # "путина_выступление" → выступления Путина
    15: 11,      # "военные_военные действия" → спецоперация
    16: 60,      # "газопровода_взрыв" → Северные потоки
    17: 380,     # "бахмут_обстрелы" → другие события в России
    18: 11,      # "войну_войне_про войну" → спецоперация
    19: 250,     # "турции_землетрясения" → землетрясение
    20: 250,     # "сирии_турции" → землетрясение
    21: 510,     # "смотрю не_не смотрю" → затрудняюсь ответить
    22: 250,     # "турции землетрясение" → землетрясение
    23: 11,      # "спецоперация_спецоперация" → спецоперация
    24: 510,     # "никаких_никаких" → затрудняюсь ответить
    25: 11,      # "спецоперация на" → спецоперация
    26: 510,     # "новости не_смотрю новости" → затрудняюсь ответить
    27: 11,      # "украине землетрясение_турции война" → выбираем спецоперацию (основное)
    28: 510,     # "не слежу_слежу" → затрудняюсь ответить
    29: 510,     # "чего_ни" → затрудняюсь ответить
    30: 250,     # "землятрясение" → землетрясение
    31: 510,     # "телевизор_смотрю телевизор" → затрудняюсь ответить
    32: 40,      # "цен_пенсии" → работа властных структур
    33: 250,     # "турции землятресение" → землетрясение
    34: 510,     # "незнаю_незнаю" → затрудняюсь ответить
    35: 510,     # "не могу_могу_сказать" → затрудняюсь ответить
    36: 510,     # "помню_не помню" → затрудняюсь ответить
    37: 16,      # "наши_наступают" → успехи армии
    38: 510,     # "таких_нет таких" → затрудняюсь ответить
    39: 11,      # "украина землетрясение" → выбираем спецоперацию
    40: 11,      # "донбасс_донбассе" → спецоперация
    41: 11,      # "стране_фронта_все_борются" → спецоперация
    42: 40,      # "политика_выступал" → работа властных структур
}

# Применяем маппинг с одним кодом
predicted_labels_single = []
for topic in topics:
    predicted_code = manual_single_code_mapping.get(topic, 510)  # по умолчанию - затрудняюсь
    predicted_labels_single.append([predicted_code])  # оборачиваем в список для совместимости

# Функция для вычисления метрик (адаптирована для одного кода)
def calculate_metrics_single(ground_truth, predicted, code):
    tp = 0  # True Positive
    fp = 0  # False Positive
    fn = 0  # False Negative

    for i in range(len(ground_truth)):
        gt_has_code = code in ground_truth[i]
        # Теперь predicted[i] содержит только один код в списке
        pred_has_code = code in predicted[i]

        if gt_has_code and pred_has_code:
            tp += 1
        elif not gt_has_code and pred_has_code:
            fp += 1
        elif gt_has_code and not pred_has_code:
            fn += 1

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1, tp, fp, fn

# Вычисляем метрики для основных кодов
important_codes = [10, 11, 16, 20, 40, 60, 70, 90, 150, 210, 250, 380, 390, 510]

results_single = []
for code in important_codes:
    precision, recall, f1, tp, fp, fn = calculate_metrics_single(ground_truth, predicted_labels_single, code)
    results_single.append({
        'Код': code,
        'Precision': round(precision, 3),
        'Recall': round(recall, 3),
        'F1-Score': round(f1, 3),
        'TP': tp,
        'FP': fp,
        'FN': fn
    })

# Создаем DataFrame с результатами
results_single_df = pd.DataFrame(results_single)
print("МЕТРИКИ КАЧЕСТВА (ОДИН КОД НА ТЕМУ):")
print(results_single_df)

# Вычисляем микро-усредненные метрики
total_tp = sum(r['TP'] for r in results_single)
total_fp = sum(r['FP'] for r in results_single)
total_fn = sum(r['FN'] for r in results_single)

micro_precision_single = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
micro_recall_single = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
micro_f1_single = 2 * micro_precision_single * micro_recall_single / (micro_precision_single + micro_recall_single) if (micro_precision_single + micro_recall_single) > 0 else 0

print(f"\nМИКРО-УСРЕДНЕННЫЕ МЕТРИКИ (ОДИН КОД НА ТЕМУ):")
print(f"Micro Precision: {micro_precision_single:.3f}")
print(f"Micro Recall: {micro_recall_single:.3f}")
print(f"Micro F1-Score: {micro_f1_single:.3f}")

# Анализ качества по основным темам
print(f"\nАНАЛИЗ ОСНОВНЫХ ТЕМ:")
major_codes = [11, 250, 510]  # Спецоперация, Землетрясение, Затрудняюсь
for code in major_codes:
    precision, recall, f1, tp, fp, fn = calculate_metrics_single(ground_truth, predicted_labels_single, code)
    print(f"Код {code}: Precision={precision:.3f}, Recall={recall:.3f}, F1={f1:.3f}")

# Сводка по распределению предсказаний
print(f"\nРАСПРЕДЕЛЕНИЕ ПРЕДСКАЗАННЫХ КОДОВ:")
predicted_codes = [code[0] for code in predicted_labels_single]
predicted_counts = Counter(predicted_codes)
for code, count in predicted_counts.most_common():
    print(f"Код {code}: {count} документов ({count/len(predicted_codes)*100:.1f}%)")

МЕТРИКИ КАЧЕСТВА (ОДИН КОД НА ТЕМУ):
    Код  Precision  Recall  F1-Score   TP  FP  FN
0    10      0.000   0.000     0.000    0   0   0
1    11      0.846   0.910     0.877  467  85  46
2    16      0.625   0.167     0.263    5   3  25
3    20      0.000   0.000     0.000    0   0   9
4    40      0.211   0.235     0.222    4  15  13
5    60      0.724   0.750     0.737   21   8   7
6    70      0.000   0.000     0.000    0   0  29
7    90      0.000   0.000     0.000    0   0   8
8   150      0.562   0.900     0.692   18  14   2
9   210      0.000   0.000     0.000    0   0   8
10  250      0.982   0.870     0.923  275   5  41
11  380      0.250   0.259     0.255    7  21  20
12  390      0.000   0.000     0.000    0   0  16
13  510      0.990   0.960     0.975  583   6  24

МИКРО-УСРЕДНЕННЫЕ МЕТРИКИ (ОДИН КОД НА ТЕМУ):
Micro Precision: 0.898
Micro Recall: 0.848
Micro F1-Score: 0.872

АНАЛИЗ ОСНОВНЫХ ТЕМ:
Код 11: Precision=0.846, Recall=0.910, F1=0.877
Код 250: Precision=0.982, Recal